In [1]:
from nanotag.data import NanotagData, ImageFileCollection, Summary
from nanotag.image import ImageSeries, GaussianFilterSlider
from nanotag.tags import PointTags, PointTagSeries
from nanotag.timeline import Timeline, TimelineTags, EditTimelineTags
from nanotag.canvas import Canvas, ToolBox
from nanotag.tools import EditPointTags, ResetView, BoxZoomTool, PanZoomTool
from nanotag.utils import link
from nanotag.histogram import Histogram
from traitlets import directional_link

from bqplot import ColorScale, Scatter, Figure
import ipywidgets as widgets
import os
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
root_directory = '/data/'
root_directory = 'D:/data/julia/'
analysis_file = 'ml_analysis.json'
filt = '/data/*.tif'
filt = '*.tif'

In [3]:
image_file_collection = ImageFileCollection(filter=filt)

images = ImageSeries()

gaussian_filter = GaussianFilterSlider(max=10)
images.filters = [gaussian_filter]

point_tags = PointTagSeries(data_fields=('sublattice', 'intensities', 'normalized_intensities', 'mask'))
defect_tags = PointTagSeries(data_fields=('type',))

point_tags.point_tags.color_scheme = 'plasma'
defect_tags.point_tags.color_scheme = 'category'

area_tags = TimelineTags(row='area', data_fields=('area',), enable_move=False, color_scale=ColorScale(min=0, max=1))
#start_tags = TimelineTags(row='start', default_t=np.zeros((1,), dtype=np.int64), color_scale=ColorScale(min=0, max=1))

nanotag_data = NanotagData(root_directory=root_directory,
                           read_file = analysis_file,
                           write_file = analysis_file,
                           tags = {'points' : point_tags, 'defects': defect_tags, 'area':area_tags}
                           )

canvas = Canvas()
canvas.tags = [point_tags, defect_tags]
canvas.image = images

timeline = Timeline()
timeline.tags = [area_tags]

toolbox = ToolBox(canvas)
toolbox.button_width = '80px'
toolbox.tools = {'Reset' : ResetView(),
                'BoxZoom' : BoxZoomTool(),
                'Pan' : PanZoomTool(),
                'S defect' : EditPointTags(defect_tags, data_fields={'type':0}),
                'Mo defect' : EditPointTags(defect_tags, data_fields={'type':1}),
               }

link((nanotag_data, 'root_directory'), (image_file_collection, 'root_directory'))

link((image_file_collection, 'images'), (images, 'images'), check_broken=False)
link((image_file_collection, 'hash'), (nanotag_data, 'identifier'), check_broken=False)

link((images, 'length'), (timeline, 'num_frames'))

link((point_tags.point_tags, 'mask'), (point_tags.point_tags._mark, 'opacity'), check_broken=False)


link((timeline, 'frame_index'), (images, 'index'))
link((timeline, 'frame_index'), (point_tags, 'index'))
link((timeline, 'frame_index'), (defect_tags, 'index'));

def area_changed(change):
    if len(area_tags.area) > 0:
        timeline.frame_index = int(np.argmax(area_tags.area > .5))

area_tags.observe(area_changed, 'area')

#defect_tags.point_tags.color_scale.colors = ['cyan', 'lime']
defect_tags.point_tags.color_scale.min= 0
defect_tags.point_tags.color_scale.max= 10

hist1 = Histogram(height=220, min=.5, max=1.5, label='S sublattice', adjust_x_scale=False)
hist1.bins=30
hist2 = Histogram(height=220, min=.5, max=1.5, label='Mo sublattice', adjust_x_scale=False)
hist2.bins=30

def create_frame_summary(i):
    summary = {}
    summary['frame_index'] = i
    summary['lattice_area_fraction'] = 0.
    summary['defect_type'] = []
    summary['defect_x'] = []
    summary['defect_y'] = []
    
    if len(area_tags.area) == 0:
        return summary
    
    summary['lattice_area_fraction'] = area_tags.area[i]
    
    try:
        summary['sampling'] = nanotag_data.selected_data['sampling'][str(i)]
    except KeyError:
        summary['sampling'] = None
    
    if not i in defect_tags.series.keys():
        return summary
    
    types = defect_tags.series[i]['type']
    summary['defect_type'] = ['Mo' if t else 'S' for t in types]
    summary['defect_x'] = defect_tags.series[i]['x']
    summary['defect_y'] = defect_tags.series[i]['y']
    return summary

def create_series_summary():
    summary = {}
    summary['md5'] = image_file_collection.hash
    summary['num_frames'] = images.length
    summary['shape_x'] = images.images.shape[1]
    summary['shape_y'] = images.images.shape[2]
    summary['boundary'] = 3.15 / np.sqrt(3) / 2
    summary['path'] = image_file_collection.relative_path
    return summary

def update_func():
    
    frame_summary = create_series_summary()
    
    frame_summary.update(create_frame_summary(timeline.frame_index))
    
    summary = create_series_summary()
    for i in range(len(defect_tags.series)):
        #summary[i] = {'frame' : {}}
        summary[i] = create_frame_summary(i)
    
    return frame_summary, summary


summary = Summary(update_func, write_file='summary.json')

def update_button(*args):
    defect_tags.update_series(timeline.frame_index)
    summary._update()

summary._update_button.on_click(update_button)

def transform_summary_path(x):
    return os.path.splitext(x)[0] + '.json'

def transform_frame_summary_path(x):
    return os.path.splitext(x)[0] + f'_{timeline.frame_index:03d}.json'

def set_current_path():
    summary.current_write_file = transform_frame_summary_path(image_file_collection.path)

timeline.observe(lambda *args: set_current_path() , 'frame_index')

directional_link((image_file_collection, 'path'), (summary, 'current_write_file'), transform_frame_summary_path)

directional_link((image_file_collection, 'path'), (summary, 'write_file'), transform_summary_path)

defect_tags.observe(lambda *args: summary._update(), 'series')

def threshold_changed(*args):
    series = {}

    for key, value in point_tags.series.items():
        
        series[key] = {'x':[],'y':[],'type':[]}
        if area_tags.area[key] < .5:
            continue
        
        mask = np.array(value['mask'], dtype=bool)
        x = np.array(value['x'])
        y = np.array(value['y'])
        sublattice = np.array(value['sublattice'])
        normalized_intensities = np.array(value['normalized_intensities'])
        
        is_under = np.zeros(len(normalized_intensities), dtype=bool)
        
        is_under[sublattice == 0] = normalized_intensities[sublattice == 0] < hist1.lower
        is_under[sublattice == 1] = normalized_intensities[sublattice == 1] < hist2.lower
        
        if sum(is_under) == 0:
            continue
        
        series[key]['x'] = x[is_under].tolist()
        series[key]['y'] = y[is_under].tolist()
        series[key]['type'] = sublattice[is_under].tolist()
    
    defect_tags.series = series
    defect_tags.update_current(defect_tags.index)
    summary._update()

def update_histrograms(change):
    hist1.sample = point_tags.point_tags.normalized_intensities[point_tags.point_tags.sublattice == 0]
    hist2.sample = point_tags.point_tags.normalized_intensities[point_tags.point_tags.sublattice == 1]
    threshold_changed()


    
point_tags.point_tags.observe(update_histrograms, 'normalized_intensities')
point_tags.point_tags.observe(threshold_changed, 'normalized_intensities')

hist1.observe(threshold_changed, 'lower')
hist2.observe(threshold_changed, 'lower')

histograms = widgets.VBox([hist1, hist2])

In [4]:
tools = widgets.VBox([nanotag_data, image_file_collection, images, gaussian_filter, point_tags, defect_tags, toolbox])

tab = widgets.Tab([tools, histograms])
tab.set_title(0, 'Tools')
tab.set_title(1, 'Histograms')

app = widgets.HBox([widgets.VBox([canvas, timeline]),
                    tab
              ,
             widgets.VBox([summary])
             
             ])

image_file_collection._load_paths()
nanotag_data.read_data()

area_changed(None)
summary._update()
app

In [14]:
from nanotag.events import KeyEvents

events = KeyEvents(app)


def toggle_point_visible(*args):
    point_tags.point_tags.visible = not point_tags.point_tags.visible

def toggle_defects_visible(*args):
    defect_tags.point_tags.visible = not defect_tags.point_tags.visible

    
events.callbacks = {'ArrowLeft' : timeline.previous_frame,
                    'ArrowRight' : timeline.next_frame,
                    'ArrowUp' : image_file_collection.next_file,
                    'ArrowDown' : image_file_collection.previous_file,
                    'r' : canvas.reset,
                    'z' : lambda : toolbox.toggle_tool('BoxZoom'),
                    'p' : lambda : toolbox.toggle_tool('Pan'),
                    'l' : toggle_point_visible,
                    'd' : toggle_defects_visible,
                   }